In [1]:
# Part 1: Import Libraries and Define the Cage Class
import copy
import random
from math import prod
from time import sleep
from datetime import datetime
from collections import defaultdict

class Cage:
    def __init__(self, cells, operation, target):
        self.cells = cells
        self.operation = operation
        self.target = target
    
    def __str__(self):
        cells = " ".join([f"({cell[0]}, {cell[1]})" for cell in self.cells])
        return f"operation -> {self.operation} / cells -> {cells} / target -> {self.target}"


In [2]:
# Part 2: Function to Generate KenKen Puzzle

def generate_KenKen(size):
    empty_grid = [[0]*size for _ in range(size)]
    final_grid = copy.deepcopy(empty_grid)
    final_grid = fill_grid_with_numbers(empty_grid, size)

    cages = generate_random_cages(final_grid, size)

    for i in range(size):
        print(final_grid[i])

    return empty_grid, cages


# Part 3: Function to Fill Grid with Numbers
def fill_grid_with_numbers(grid, size):
    """
    This function fills the grid with numbers 1 to size such that
    each row and each column has unique values. It uses backtracking
    to ensure all cells are filled with valid numbers.
    """

    def is_safe_to_place(grid, row, col, num):
        # Check row and column uniqueness
        check = True
        for i in range(size):
            if grid[row][i] == num or grid[i][col] == num:
                check = False
                break

        return check
    
    def get_empty_cell_or_none(grid):
        for i in range(size):
            for j in range(size):
                if grid[i][j] == 0:
                    return (i, j)
        return (-1, -1)


    def fill_grid_backtracking(grid, row, col):
        # Fills the grid recursively with valid numbers using backtracking
        if row == -1 and col == -1:
            return True, grid
        cur_grid = copy.deepcopy(grid)
        values = [i for i in range(1, size+1)]
        random.shuffle(values)
        for val in values:
            if(is_safe_to_place(grid, row, col, val)):
                cur_grid[row][col] = val
                empty_row, empty_col = get_empty_cell_or_none(cur_grid)
                created, res_grid = fill_grid_backtracking(cur_grid, empty_row, empty_col)
                if created:
                    return created, res_grid

        return False, grid

    # Start backtracking from the first cell
    _ , returned_grid = fill_grid_backtracking(grid, 0, 0)
    return returned_grid


# Part 4: Function to Generate Random Cages

def generate_random_cages(grid, size):
    cages = []
    visited = [[False] * size for _ in range(size)]

    for i in range(size):
        for j in range(size):
            if not visited[i][j]:
                cage_size = random.randint(1, size)
                cells = [(i, j)]
                visited[i][j] = True

                while len(cells) < cage_size:
                    x, y = cells[-1]
                    neighbors = [(x + dx, y + dy) for dx, dy in [(1, 0), (0, 1), (-1, 0), (0, -1)]
                                 if 0 <= x + dx < size and 0 <= y + dy < size and not visited[x + dx][y + dy]]
                    if neighbors:
                        next_cell = random.choice(neighbors)
                        cells.append(next_cell)
                        visited[next_cell[0]][next_cell[1]] = True
                    else:
                        break

                operation = random.choice(['+', '*']) if cage_size > 2 else random.choice(['+', '-', '*', '/'])
                target = calculate_target(grid, cells, operation)
                cages.append(Cage(cells, operation, target))

    return cages


# Function to calculate the target based on the operation for a given cage
def calculate_target(grid, cells, operation):
    assert len(cells) >= 1, "every cage must have at least one cell"
    assert len(cells) <= 2 or operation in ["+", "*"], "'-' and '/' are available for cages with length at most 2"
    
    cell_values = [grid[cell[0]][cell[1]] for cell in cells]

    if len(cells) == 1:
        return cell_values[0]
    elif operation == "-":
        return max(cell_values) - min(cell_values)
    elif operation == "/":
        return max(cell_values) / min(cell_values)
    elif operation == "+":
        return sum(cell_values)
    elif operation == "*":
        return prod(cell_values)



In [3]:
# Part 5: KenKen Backtracking Solver

def solve_kenken_backtracking(grid, cages, row, col):
    # sleep(0.02)
    # for i in range(len(grid)):
    #     print(grid[i])
    # print("-"*20)
        
    if row == -1 and col == -1:
        return True, grid
    
    size = len(grid)
    all_vals = set([i for i in range(1, size+1)])
    # used_vals = set()
    # for i in range(size):
    #     if grid[row][i] != 0:
    #         used_vals.add(grid[row][i])
    #     if grid[i][col] != 0:
    #         used_vals.add(grid[i][col])
    # available_vals = all_vals - used_vals
    cur_grid = copy.deepcopy(grid)
    for value in all_vals:
        cur_grid[row][col] = value
        if is_safe_kenken(cur_grid, row, col, value, cages):
            empty_row, empty_col = find_unassigned_location(cur_grid)
            valid, res_grid = solve_kenken_backtracking(cur_grid, cages, empty_row, empty_col)
            if valid:
                return True, res_grid
    
    return False, grid
            

def solve_kenken(grid, cages):
    start = datetime.now()
    _, final_grid = solve_kenken_backtracking(grid, cages, 0, 0)
    end = datetime.now()
    # print(final_grid)
    return final_grid, end-start



def is_safe_kenken(grid, row, col, num, cages):
    # Check row, column numbers uniqueness
    size = len(grid)
    is_unique = True
    for i in range(size):
        if (grid[row][i] == num and i != col) or (grid[i][col] == num and i != row):
            is_unique = False
            break
    
    cage_check = True
    for cage in cages:
        values = [grid[cell[0]][cell[1]] for cell in cage.cells]
        check = validate_cage_operation(cage.operation, values, cage.target)
        if check is False:
            cage_check = False
            # print(f"bad cage: {cage}")
            # print(f"current grid: {grid}")
            # print("*"*20)
            break
    
    return (is_unique and cage_check)

def validate_cage_operation(operation, values, target):
    # Check if the operation on the cage values matches the target
    if 0 in values:
        return None
    if len(values) == 1:
        return True
    if operation == '+':
        return sum(values) == target
    elif operation == "*":
        return prod(values) == target
    elif operation == "/":
        return max(values) / min(values) == target
    elif operation == "-":
        return max(values) - min(values) == target
    else:
        return False

def find_unassigned_location(grid):
    # Find the first empty cell in the grid
    size = len(grid)
    for i in range(size):
        for j in range(size):
            if grid[i][j] == 0:
                return (i, j)
    return (-1, -1)



In [4]:
# Part 6: KenKen Domain Constraint Solver

def solve_kenken_csp(grid, cages):
    """
    Function to solve the KenKen grid using Constraint Satisfaction Problem (CSP)
    """
    start = datetime.now()
    size = len(grid)
    def create_domains(grid):
        """
        Function to create domains for each cell in the grid
        """
        domains = dict()
        default_domains = set([i for i in range(1, size+1)])
        for i in range(size):
            for j in range(size):
                # cause values are int, it is okay to shallow copy them
                domains[(i, j)] = default_domains.copy()
        return domains
    

    def validate_cage_operation(operation, domains, target):
        # Check if the operation on the cage values matches the target
        for domain in domains.values():
            if len(domain) > 1:
                return None
            
        if len(domains) == 1:
            for key, value in domains.items():
                return list(value)[0] == target
        
        values = [list(s)[0] for s in domains.values()]
        if operation == '+':
            return sum(values) == target
        elif operation == "*":
            return prod(values) == target
        elif operation == "/":
            return max(values) / min(values) == target
        elif operation == "-":
            return max(values) - min(values) == target
        else:
            return False
        

    def is_valid_assignment(i, j, val, assignment, cages):
        """
        Function to check if assigning a value to a cell is valid
        Using Forward Checking
        """
        
        domain = assignment[(i, j)]
        if val not in domain:
            return False, assignment

        new_assignment = copy.deepcopy(assignment)
        new_assignment[(i, j)] = set([val])
        for itr1 in range(size):
            if itr1 != j:
                new_assignment[(i, itr1)].discard(val)
                if not new_assignment[(i, itr1)]:
                    return False, new_assignment
            if itr1 != i:
                new_assignment[(itr1, j)].discard(val)
                if not new_assignment[(itr1, j)]:
                    return False, new_assignment
        
        for cage in cages:
            domains = {(i, j): new_assignment[(i, j)] for (i, j) in cage.cells}
            if validate_cage_operation(cage.operation, domains, target=cage.target) is False:
                return False, new_assignment

        return True, new_assignment

    def find_unassigned_location(assignment):
        """
        Function to find an unassigned location in the grid
        """
        for (i, j), value in assignment.items():
            if len(value) > 1:
                return (i, j)
            
        return (-1, -1)

    def solve_csp(assignment):
        """
        Recursive function to solve grid with CSP
        """
        # for i in range(size):
        #     for j in range(size):
        #         print("{:<10}".format(f"{assignment[(i, j)]}"), end=" ")
        #     print("\n")
        # print("*"*100)

        all_values = [i for i in range(1, size+1)]
        # all_values.shuffle()
        row, col = find_unassigned_location(assignment)
        if (row, col) == (-1, -1):
            return True, assignment
        for value in all_values:
            is_valid, new_assignment = is_valid_assignment(row, col, value, assignment, cages)
            if is_valid:
                result, final_assignment = solve_csp(new_assignment)
                if result:
                    return True, final_assignment
        
        return False, assignment


    # Create initial domains for each cell in the grid
    domains = create_domains(grid)
    assignment = {(i, j): val for (i, j), val in domains.items()}

    # Solve the KenKen grid using CSP
    solved, final_assignment = solve_csp(assignment)
    end = datetime.now()
    if solved:
        solved_grid = [[final_assignment[(i, j)] for j in range(len(grid))] for i in range(len(grid))]
        return solved_grid, end-start
    else:
        return None, end-start


In [5]:
# Part 7: Print Solution

def print_solution(grid):
    for row in grid:
        print(" ".join(str(x) for x in row))


In [6]:
# Part 8: Run Example
def solve(size):
    unsolved_grid, kenken_cages = generate_KenKen(size)

    print("Generated KenKen Cages:")
    for cage in kenken_cages:
        print(f"Cage Cells: {cage.cells}, Operation: {cage.operation}, Target: {cage.target}")
    print("-----------------------")

    # Solve using Backtracking solver
    backtrack_grid = copy.deepcopy(unsolved_grid)
    result, duration_bt = solve_kenken(backtrack_grid, kenken_cages)
    if result:
        print("Solved KenKen Puzzle (BackTracking):")
        print_solution(result)
        print(f"{duration_bt=}")
    else:
        print("No solution found using BackTracking.")
    print("-----------------------")


    # Solve using domain CSP solver
    csp_solved, duration_csp = solve_kenken_csp(unsolved_grid, kenken_cages)
    if csp_solved:
        print("Solved KenKen Puzzle (domain CSP):")
        print_solution(csp_solved)
        print(f"{duration_csp=}")
    else:
        print("No solution found using CSP.")
    
    return duration_bt.total_seconds(), duration_csp.total_seconds()


In [7]:
# first number in tuple indicates size of kenken and second one indicates repetions of run
# for example (5, 3) means running solve method for size = 5 for 3 times
size_rep = [(4, 3), (5, 3), (6, 3)]


size_to_exectime = defaultdict(dict)
for size, rep in size_rep:
    size_to_exectime[size] = {"bt": [], "csp": []}
    for _ in range(rep):
        bt, csp = solve(size)
        size_to_exectime[size]["bt"].append(bt)
        size_to_exectime[size]["csp"].append(csp)


[3, 4, 2, 1]
[2, 1, 3, 4]
[1, 3, 4, 2]
[4, 2, 1, 3]
Generated KenKen Cages:
Cage Cells: [(0, 0), (1, 0)], Operation: -, Target: 1
Cage Cells: [(0, 1), (0, 2), (1, 2)], Operation: *, Target: 24
Cage Cells: [(0, 3)], Operation: -, Target: 1
Cage Cells: [(1, 1)], Operation: +, Target: 1
Cage Cells: [(1, 3)], Operation: +, Target: 4
Cage Cells: [(2, 0), (3, 0), (3, 1), (2, 1)], Operation: +, Target: 10
Cage Cells: [(2, 2), (3, 2), (3, 3)], Operation: +, Target: 8
Cage Cells: [(2, 3)], Operation: *, Target: 2
-----------------------
Solved KenKen Puzzle (BackTracking):
2 3 4 1
1 4 2 3
4 1 3 2
3 2 1 4
duration_bt=datetime.timedelta(microseconds=4620)
-----------------------
Solved KenKen Puzzle (domain CSP):
{2} {4} {3} {1}
{3} {1} {2} {4}
{1} {3} {4} {2}
{4} {2} {1} {3}
duration_csp=datetime.timedelta(microseconds=8220)
[4, 3, 2, 1]
[3, 4, 1, 2]
[1, 2, 4, 3]
[2, 1, 3, 4]
Generated KenKen Cages:
Cage Cells: [(0, 0), (1, 0), (2, 0), (3, 0)], Operation: +, Target: 10
Cage Cells: [(0, 1), (1, 1

KeyboardInterrupt: 

In [29]:
from statistics import mean, variance

formatted_string = "{:^3} | {:^8} | {:^35} | {:^8} | {:^8}"
header = formatted_string.format("n", "method", "exec times", "avg", "variance")
print(header, "\n", "-"*len(header))
for size, exex_list in size_to_exectime.items():
    formatted_string = "{:^3} | {:^8} | {:^35} | {:.6f} | {:.6f}"
    print(formatted_string.format(size, "bt", f"{exex_list["bt"]}", mean(exex_list["bt"]), variance(exex_list["bt"])))
    print(formatted_string.format(size, "csp", f"{exex_list["csp"]}", mean(exex_list["csp"]), variance(exex_list["csp"])))

 n  |  method  |             exec times              |   avg    | variance 
 --------------------------------------------------------------------------
 4  |    bt    |   [0.002281, 0.000544, 0.003689]    | 0.002171 | 0.000002
 4  |   csp    |   [0.035835, 0.007741, 0.010513]    | 0.018030 | 0.000240
 5  |    bt    |   [0.280149, 0.003981, 0.290589]    | 0.191573 | 0.026420
 5  |   csp    |   [0.016697, 0.040149, 0.231663]    | 0.096170 | 0.013906
 6  |    bt    |   [10.934832, 1.578628, 7.474251]   | 6.662570 | 22.378757
 6  |   csp    |   [4.843814, 8.097457, 135.78325]   | 49.574840 | 5576.563971
